In [1]:
# RAWDATA_BEETLE_20210616_CS.xlsx
from google.colab import files
uploaded = files.upload()

Saving RAWDATA_BEETLE_20210616_CS.xlsx to RAWDATA_BEETLE_20210616_CS.xlsx


In [96]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import preprocessing
import urllib
import cv2

from google.colab.patches import cv2_imshow

pd.options.display.float_format = '{:,.2f}'.format
def format_float(value):
    return f'{value:,.2f}'
pd.options.display.float_format = format_float

In [97]:
df = pd.read_excel([*uploaded][0])

In [98]:
column_name = list(df.columns)

In [99]:
column_name = [sub.replace('Dalam satu bulan, berapa rata - rata total uang yang dibelanjakan untuk produk yang dijual di toko melalui',
                           'Belanja Produk yg dijual melalui').
               replace('Platform online atau Kemitraan apa yang sudah digunakan oleh toko untuk penyedia layanan transfer/ kirim uang?:',
                           'Layanan Keuangan_').
               replace('Platform Online atau Kemitraan apa yang sudah digunakan oleh toko untuk penyediaan layanan jasa seperti jual pulsa, bayar tagihan, pembelian token, pembelian tiket, tabungan emas, zakat, dll?:',
                           'Layanan Jasa_').
               replace('Platform online atau Kemitraan apakah yang sudah digunakan oleh toko untuk penyediaan produk kebutuhan sehari (makanan, minuman, rumah tangga, dll)?:',
                           'Layanan Groceries_').
               replace('Jika dual bisnis, apa saja bisnisnya?:',
                           'Dual_').
               replace('Brand Platform Online apa yang ada di penanda toko tersebut?:',
                           'Branding_').
               replace('Mengapa Anda belum/tidak menjadi Agen BukaSend di aplikasi Mitra Bukalapak?:',
                           'Alasan_').
               replace('Informasi apa sajakah yang ingin Anda ketahui tentang Agen BukaSend?:',
                           'Informasi_').
               replace('Melalui Media/channel apakah yang Anda sukai untuk mendapatkan info terbaru seputar Agen BukaSend?:',
                           'Info_').
               replace('Apakah pembeli bisa melakukan transaksi digital melalui platform berikut:',
                           'Transaksi melalui_').
               replace('Ambil Foto penanda toko/papan nama toko/spanduk toko yang terdapat Branding Online!',
                       'Foto Branding Online').
               replace('Ambil Foto penanda toko/papan nama toko/spanduk toko!',
                       'Foto Nama Toko').
               replace('Apakah toko ini dual bisnis?',
                       'Dual Bisnis').
               replace('Apakah toko mempunyai keinginan untuk bergabung sebagai Mitra Bukalapak?',
                       'Ingin jadi Mitra Bukalapak').
               replace('Apakah Anda tahu dan/atau pernah menjadi Agen BukaSend?',
                       'Tahu/pernah jadi Agen BukaSend').
               replace('Apakah pemilik/penjaga toko memperbolehkan untuk mengambil foto tampak dalam toko?',
                       'Diijinkan mengambil foto').
               replace('Apakah ada penanda toko permanen yang terlihat di luar toko yang didominasi Brand Platform Online tertentu? (SA)',
                       'Branding Platform Online di Nama Toko').
               replace('Apakah ada penanda toko/papan nama toko/spanduk toko?',
                       'Ada Papan Nama Toko').
               replace('Lokasi Toko - (Pilih yang terdekat)',
                       'Lokasi Toko').
               replace('Ukuran Luas Toko (m2)',
                       'Luas Toko(m2)').
               replace('Apakah toko sudah menggunakan platform online untuk bisnisnya? (Platform Online yang dimaksud adalah toko menjadi mitra dengan instansi tertentu yang akan menjadi penyalur/penyedia produk2 yang akan dijual)',
                       'Ada Platform Online').
               replace('Seberapa tertarik atau tidak tertarikkah Anda untuk menjadi Agen BukaSend?',
                       'Tertarik jadi Agen BukaSend').
               replace('Sebutkan Media/channel apakah yang Anda sukai untuk mendapatkan info terbaru seputar Agen BukaSend lainya?',
                       ' Media info seputar Agen BukaSend').
               replace('Silahkan ambil foto tampak dalam Toko (- untuk Toko pulsa, unjung ke ujung etalase - untuk GT di utmakan jika ada brand online platform, di foto)',
                       'Foto dalam Toko').
               replace('Mengambil foto tampak luar toko (Pastikan dapat memepermudah mengidentifikasi bangunan, jarak sekitar max 5 s/d 7 meter)',
                       'Foto luar Toko').
               replace('Apakah toko menggunakan internet (untuk oerasional/ kegiatan toko) ?',
                       'Toko menggunakan internet').
               replace('Jam buka Operasional (00.00-23.59)',
                       'Jam Buka').
               replace('Jam tutup Operasional (00.00-23.59)',
                       'Jam Tutup').
               replace('(Q9A - Q9B )',
                       'Jam Operasional').
               replace('Estimasi OMSET TOKO (penjualan toko) PER HARI? (dalam Rupiah)',
                       'OMSET PER HARI').
               replace('Estimasi OMSET TOKO (penjualan toko) PER MINGGU (dalam Rupiah)',
                       'OMSET PER MINGGU').
               replace('Sudah berapa lama toko beroperasi?',
                       'Usia Toko').
               replace('Berapakah jumlah pegawai toko?',
                       'Jmlh pegawai').
               replace('Device / alat apa yang biasa toko gunakan untuk mengakses internet?:',
                       'Device utk internet').
               replace('Berapakah perkiraan Range usia Pemilik Toko?',
                       'Usia OWNER').
               replace('Nama Jalan Alamat Toko',
                       'Alamat').
               replace('Nomor Rumah / Bangunan (sesuai alamat )',
                       'Nomor').
               replace("RT - Maximal 3 digit Di awali dari '0' misal (001)",
                       'RT').
               replace("RW - Maximal 3 digit Di awali dari '0' misal (001)",
                       'RW').
               replace('Ukuran Jalan', ' Ukuran Jalan').
               replace("'Sebutkan sumber pembelian supply toko:",
                       'Sumber supply_')                                         
                for sub in column_name]

In [100]:
new_name = []
i = 0
for text in column_name:
  if text.find('.') == -1:
    new_name.append(text)
    continue
  else:
    ind = text.find('.')
    new_name.append(text[ind+2:])
    i += 1

#for i in range(len(new_name)):
#  print(i , new_name[i])

In [101]:
new_name[154:169] =  [sub.replace('Belanja Produk yg dijual melalui',
                                  'Hasil dari Layanan Jasa:') for sub in new_name[154:169]]
new_name[187:204] =  [sub.replace('Belanja Produk yg dijual melalui',
                                  'Hasil dari Layanan Keuangan:') for sub in new_name[187:204]]

In [102]:
for i in range(len(new_name)):
  print(i , new_name[i])

0 RESPONSE_ID
1 Tanggal Survey
2 PROVINCE
3 KOTA/KABUPATEN
4 KECAMATAN
5 KELURAHAN
6 Zipcode
7 Ukuran Jalan
8 Lokasi Toko
9 Lokasi Toko Lainnya..
10 Infomasi tambahan lokasi toko
11 Ada Papan Nama Toko
12 Nama Toko
13 Branding Platform Online di Nama Toko
14 Branding_Ayo SRC
15 Branding_Ayopop
16 Branding_BCA M-Banking
17 Branding_Blibli Mitra
18 Branding_BNI/Syariah Mobile Banking
19 Branding_BNI46
20 Branding_BRIlink
21 Branding_Dana
22 Branding_Doku
23 Branding_Flip
24 Branding_Gopay
25 Branding_GrabKios
26 Branding_Gudang Pulsa
27 Branding_GudangAda
28 Branding_Indopay
29 Branding_iSaku
30 Branding_JakOne Mobile
31 Branding_Kioser
32 Branding_LangitPay
33 Branding_LinkAja
34 Branding_Mitra Bukalapak
35 Branding_GoToko
36 Branding_Mitra Shopee
37 Branding_Mitra Tokopedia
38 Branding_Mkios
39 Branding_MPI Reload
40 Branding_OCTO Go Mobile
41 Branding_Ovo
42 Branding_OBA ( Outlet Binaan Alfamart)
43 Branding_Payfazz
44 Branding_Paytren
45 Branding_PermataMe
46 Branding_Propana Reload


In [103]:
df.columns = new_name

In [104]:
df.describe()

,RESPONSE_ID,Zipcode,Luas Toko(m2),RW,Nomor Lantai (hanya untuk yang berada di dalam bangunan berdasarkan jawaban Q2),Lot/Los/Blok (hanya untuk yang berada di dalam bangunan berdasarkan jawaban Q2),Alasan menolak Lainnya...,Nomor Telepon - Telkom (Kode Area + Nomor Telepon),Nomor Handphone Toko,Nomor Handphone Pemilik Toko,Jam Buka,Jam Tutup,Jam Operasional,Alasan Toko buka Kurang dari 6 Jam,OMSET PER HARI,Calculate Estimasi OMSET TOKO (penjualan toko) PER HARI X 7,OMSET PER MINGGU,"Estimasi OMSET TOKO (penjualan toko Harian), PER Bulan (dalam Rupiah)","Estimasi OMSET TOKO (penjualan toko mingguan) ,PER Bulan (dalam Rupiah)",Belanja Produk yg dijual melalui Ayo SRC,Belanja Produk yg dijual melalui Blibli Mitra,Belanja Produk yg dijual melalui GudangAda,Belanja Produk yg dijual melalui Mitra Bukalapak,Belanja Produk yg dijual melalui Mitra Tokopedia,Belanja Produk yg dijual melalui Ula,Belanja Produk yg dijual melalui Warung Pintar,Belanja Produk yg dijual melalui Go Toko,Belanja Produk yg dijual melalui OBA ( Outlet Binaan Alfamart),Hasil dari Layanan Jasa: Ayopop,Hasil dari Layanan Jasa: Blibli Mitra,Hasil dari Layanan Jasa: GrabKios,Hasil dari Layanan Jasa: Gudang Pulsa,Hasil dari Layanan Jasa: Indopay,Hasil dari Layanan Jasa: Kioser,Hasil dari Layanan Jasa: LangitPay,Hasil dari Layanan Jasa: LinkAja,Hasil dari Layanan Jasa: Mitra Bukalapak,Hasil dari Layanan Jasa: Mitra Shopee,Hasil dari Layanan Jasa: Mitra Tokopedia,Hasil dari Layanan Jasa: Mkios,Hasil dari Layanan Jasa: MPI Reload,Hasil dari Layanan Jasa: Payfazz,Hasil dari Layanan Jasa: Propana Reload,Hasil dari Layanan Keuangan: BCA M-Banking,Hasil dari Layanan Keuangan: BNI/Syariah Mobile Banking,Hasil dari Layanan Keuangan: BNI46,Hasil dari Layanan Keuangan: BRIlink,Hasil dari Layanan Keuangan: Dana,Hasil dari Layanan Keuangan: Doku,Hasil dari Layanan Keuangan: Flip,Hasil dari Layanan Keuangan: Gopay,Hasil dari Layanan Keuangan: iSaku,Hasil dari Layanan Keuangan: JakOne Mobile,Hasil dari Layanan Keuangan: Mitra Bukalapak (KirimUang),Hasil dari Layanan Keuangan: OCTO Go Mobile,Hasil dari Layanan Keuangan: Ovo,Hasil dari Layanan Keuangan: Payfazz,Hasil dari Layanan Keuangan: Paytren,Hasil dari Layanan Keuangan: PermataMe,Hasil dari Layanan Keuangan: Sakuku
count,"3,042.00","3,042.00","3,042.00","3,042.00",21.00,21.00,0.00,35.00,"1,426.00","1,555.00","3,038.00","3,038.00","3,038.00",0.00,"2,862.00","2,862.00",176.00,"2,862.00",176.00,88.00,19.00,1.00,226.00,73.00,10.00,42.00,0.00,0.00,3.00,6.00,10.00,1.00,0.00,0.00,8.00,53.00,196.00,128.00,55.00,81.00,12.00,16.00,3.00,33.00,2.00,14.00,81.00,26.00,1.00,0.00,27.00,2.00,0.00,89.00,0.00,38.00,7.00,2.00,1.00,0.00
mean,"18,266,496.61","36,742.28",12.12,131.28,571.71,390.38,nan,"875,345,954.77","89,211,100,738.68","106,074,732,760.50",7.18,21.32,14.49,nan,"644,558.00","4,722,835.45","2,136,647.73","18,050,796.72","8,587,500.00","2,864,090.91","2,216,315.79","3,000,000.00","2,249,513.27","2,102,054.79","2,335,000.00","1,648,809.52",nan,nan,"6,000,000.00","2,416,666.67","3,252,000.00","1,500,000.00",nan,nan,"3,437,500.00","2,647,169.81","2,493,673.46","2,776,679.69","2,611,090.91","3,803,086.42","1,262,500.00","3,262,500.00","4,000,000.00","4,400,000.00","1,400,000.00","3,008,571.43","3,901,358.02","2,356,153.85","3,000,000.00",nan,"2,018,518.52","3,000,000.00",nan,"3,244,382.02",nan,"2,202,631.58","3,071,428.57","1,500,000.00","10,000,000.00",nan
std,"11,800.42","21,675.39",12.06,327.95,505.57,489.35,nan,"1,453,228,163.25","96,368,321,905.97","373,406,979,317.31",1.53,2.79,2.36,nan,"951,595.93","7,444,659.58","1,897,104.74","26,643,583.46","7,562,528.87","2,783,687.14","2,537,507.99",nan,"2,414,946.84","2,333,235.69","2,963,673.58","2,192,656.48",nan,nan,"3,605,551.28","2,073,081.44","3,956,294.00",nan,nan,nan,"4,083,394.07","2,771,184.40","2,619,918.53","2,934,541.66","2,974,463.33","3,007,431.90","762,509.31","3,053,495.26","3,905,124.84","3,666,742.42","1,555,634.92","3,875,853.96","3,438,807.84","3,019,235.77",nan,nan,"2,364

In [105]:
# DROP COLUMNS:
drop_col = ['Alasan Toko buka Kurang dari 6 Jam',
            'Mitra Bukalapak adalah aplikasi yang memberikan kesempatan kepada semua masyarakat yang memiliki usaha maupun tidak untuk mendapatkan penghasilan tambahan dengan berjualan produk grosir dan/atau produk virtual. ',
            'Agen BukaSend adalah penghubung antara Pelanggan yang ingin mengirimkan barang, dengan kurir logistik yang telah bekerjasama. Saat ini, kurir yang telah bekerja sama dengan fitur Agen BukaSend yaitu : SiCepat Ekspres, J&T Express, Lion Parcel, JNE, Ninja Xpress dan Grab. Mitra dimana Agen Buka Send bisa dapat cashback ongkos kirim hingga 35%.',
            'Anda tidak perlu mengeluarkan biaya untuk menjadi Agen BukaSend, hanya cukup melakukan pendaftaran di aplikasi Mitra Bukalapak, dan siap menerima siapa saja yang ingin melakukan pengiriman barang',
            'STORE_STATUS', 'Alamat', 'Nomor', 'RT', 'RW', 'Nama responden', 'Nama Lengkap Pemilik Toko',
            'Infomasi tambahan lokasi toko', 'Tanggal Survey', 'Lokasi Toko Lainnya..'
            ]
df = df.drop(drop_col, axis = 1)


In [106]:
for i in range(len(df.columns)):
  print(i , df.columns[i])

0 RESPONSE_ID
1 PROVINCE
2 KOTA/KABUPATEN
3 KECAMATAN
4 KELURAHAN
5 Zipcode
6 Ukuran Jalan
7 Lokasi Toko
8 Ada Papan Nama Toko
9 Nama Toko
10 Branding Platform Online di Nama Toko
11 Branding_Ayo SRC
12 Branding_Ayopop
13 Branding_BCA M-Banking
14 Branding_Blibli Mitra
15 Branding_BNI/Syariah Mobile Banking
16 Branding_BNI46
17 Branding_BRIlink
18 Branding_Dana
19 Branding_Doku
20 Branding_Flip
21 Branding_Gopay
22 Branding_GrabKios
23 Branding_Gudang Pulsa
24 Branding_GudangAda
25 Branding_Indopay
26 Branding_iSaku
27 Branding_JakOne Mobile
28 Branding_Kioser
29 Branding_LangitPay
30 Branding_LinkAja
31 Branding_Mitra Bukalapak
32 Branding_GoToko
33 Branding_Mitra Shopee
34 Branding_Mitra Tokopedia
35 Branding_Mkios
36 Branding_MPI Reload
37 Branding_OCTO Go Mobile
38 Branding_Ovo
39 Branding_OBA ( Outlet Binaan Alfamart)
40 Branding_Payfazz
41 Branding_Paytren
42 Branding_PermataMe
43 Branding_Propana Reload
44 Branding_Sakuku
45 Branding_Ula
46 Branding_Warung Pintar
47 Foto Brandin

In [107]:
print(df.shape)
df.head()

(3042, 234)


,RESPONSE_ID,PROVINCE,KOTA/KABUPATEN,KECAMATAN,KELURAHAN,Zipcode,Ukuran Jalan,Lokasi Toko,Ada Papan Nama Toko,Nama Toko,Branding Platform Online di Nama Toko,Branding_Ayo SRC,Branding_Ayopop,Branding_BCA M-Banking,Branding_Blibli Mitra,Branding_BNI/Syariah Mobile Banking,Branding_BNI46,Branding_BRIlink,Branding_Dana,Branding_Doku,Branding_Flip,Branding_Gopay,Branding_GrabKios,Branding_Gudang Pulsa,Branding_GudangAda,Branding_Indopay,Branding_iSaku,Branding_JakOne Mobile,Branding_Kioser,Branding_LangitPay,Branding_LinkAja,Branding_Mitra Bukalapak,Branding_GoToko,Branding_Mitra Shopee,Branding_Mitra Tokopedia,Branding_Mkios,Branding_MPI Reload,Branding_OCTO Go Mobile,Branding_Ovo,Branding_OBA ( Outlet Binaan Alfamart),...,Ingin jadi Mitra Bukalapak,Tahu/pernah jadi Agen BukaSend,Tertarik jadi Agen BukaSend,Alasan_Belum Paham apa itu BukaSend,Alasan_Belum mendapatkan pelanggan yang mau mengirim barang,Alasan_Belum percaya dengan BukaSend,Alasan_Belum siap menambah kegiatan bisnis baru,Alasan_Selain di atas,Informasi_Bagaimana cara melakukan pemesanan transaksi,Informasi_Informasi cashback,Informasi_Informasi mendapatkan pelanggan (pengirim),Informasi_Informasi meningkatkan penjualan,Informasi_Metode Pembayaran,Informasi_Selain di atas,Informasi_Syarat Menjadi Agen BukaSend,Info_Email dari aplikasi,Info_Group Komunitas,Info_Iklan dari social media (Instagram/Facebook),Info_Influencer (orang terkenal/artis/youtuber/dll),Info_Informasi dari teman,Info_Notifikasi aplikasi,Info_Poster/banner di dalam aplikasi,Info_TV,Info_Selain di atas,Media info seputar Agen BukaSend,Transaksi melalui_Dana,Transaksi melalui_Gopay,Transaksi melalui_Linkaja,Transaksi melalui_Ovo,Transaksi melalui_Qris,Transaksi melalui_Sakuku,Transaksi melalui_Shopeepay,Transaksi melalui_NA,Berapa daya listrik yang terpasang pada toko?,Diijinkan mengambil foto,Foto dalam Toko,Foto luar Toko,Geo Tag 2,Interview Status,Store Comment
0,18282112,SULAWESI UTARA,MANADO,MALALAYANG,MALALAYANG I BARAT,95163,2 Mobil,Perumahan,Ada,Haruko Cell,Ada,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,Tidak,"Pernah mendengar, tetapi tidak tahu maksud dar...",Antara tertarik dan tidak tertarik,NaN,NaN,NaN,NaN,NaN,True,False,True,False,False,False,True,False,False,True,False,False,False,False,True,False,NaN,False,False,False,False,False,False,False,True,CL 4 (900 watt),Ya,https://storage.googleapis.com/opspedia-155244...,https://storage.googleapis.com/opspedia-155244...,1.44286848;124.8082534;3.216;gps,Complete,Complete
1,18280641,JAWA BARAT,BANDUNG,RANCABALI,ALAMENDAH,40973,Lebih Dari 2 Mobil,Perumahan,Ada,Rizal Cell,Tidak Ada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Tidak tahu sama sekali,Tertarik,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,NaN,False,False,False,False,False,False,False,True,CL 4 (900 watt),Ya,https://storage.googleapis.com/opspedia-155244...,https://storage.googleapis.com/opspedia-155244...,-7.12666668;107.42038683;3.216;gps,Complete,ok
2,18280551,JAWA BARAT,BANDUNG,RANCABALI,ALAMENDAH,40973,2 Mobil,Perumahan,Ada,IBRAHIM CELL,Tidak Ada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tidak,Tidak tahu sama sekali,Tidak tertarik,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,NaN,False,False,False,False,False,False,False,True,CL 4 (900 watt),Ya,https://storage.googleapis.com/opspedia-155244...,https://storage.googleapis.com/opspedia-155244...,-7.12668734;107.42038532;3.216;gps,Complete,Ok
3,18280541,JAWA BARAT,CIREBON,LEMAHWUNGKUK,PEGAMBIRAN,45113,1 Mobil,Perumahan,Ada,Src Mun,Ada,True,False,False,False,False,False,False,False,False,False,False,False,Fa

# Provinsi dan Kota tempat pengambilan sample
Dicek ulang dengan jumlah sample masing2 area yg diharapkan

In [57]:
df.pivot_table(index=['PROVINCE','KOTA/KABUPATEN'],
               values= ['RESPONSE_ID'],
               columns= ['Type toko'],
               aggfunc= 'count')

RESPONSE_ID  ...           
Type toko                          General Trade Large  ... Kios Pulsa
PROVINCE           KOTA/KABUPATEN                       ...           
DI YOGYAKARTA      YOGYAKARTA                       10  ...         19
DKI JAKARTA        JAKARTA BARAT                    11  ...         88
                   JAKARTA PUSAT                     5  ...         35
                   JAKARTA SELATAN                   7  ...         57
                   JAKARTA TIMUR                    23  ...        204
                   JAKARTA UTARA                    30  ...         81
JAWA BARAT         BANDUNG                          38  ...        108
                   CIREBON                          24  ...         15
                   KARAWANG                         20  ...        100
                   SUKABUMI                         25  ...         16
JAWA TIMUR         SURABAYA                         23  ...        123
KALIMANTAN SELATAN BANJARMASIN                       8  ...         32
LAMPUNG            BANDAR LAMPUNG                    9  ...         43
RIAU               PEKANBARU                         6  ...         41
SULAWESI SELATAN   MAKASSAR                         11  ...         56
SULAWESI UTARA     MANADO                            8  ...         20
SUMATERA SELATAN   PALEMBANG                        10  ...         73
SUMATERA UTARA     MEDAN                            23  ...         89

[18 rows x 3 columns]

In [108]:
df['Estimasi OMSET TOKO (penjualan toko Harian), PER Bulan (dalam Rupiah)'] = df['Estimasi OMSET TOKO (penjualan toko Harian), PER Bulan (dalam Rupiah)'].fillna(0)
df['Estimasi OMSET TOKO (penjualan toko mingguan) ,PER Bulan (dalam Rupiah)'] = df['Estimasi OMSET TOKO (penjualan toko mingguan) ,PER Bulan (dalam Rupiah)'].fillna(0)

df['OMSET PER BULAN'] = df['Estimasi OMSET TOKO (penjualan toko Harian), PER Bulan (dalam Rupiah)'] + df['Estimasi OMSET TOKO (penjualan toko mingguan) ,PER Bulan (dalam Rupiah)'] 



In [121]:
conditions = [
    (df['OMSET PER BULAN'] > 10000000),
    (df['OMSET PER BULAN'] <= 10000000) & (df['OMSET PER BULAN'] >= 3000000),
    (df['OMSET PER BULAN'] < 3000000)]
              
values = ['GT Large', 'GT Small', 'GT Warung']

df['Cek Tipe GT'] = np.select(conditions, values)


In [125]:
# Cek Tipe Toko (GT Large/ GT Small/ Kios Pulsa)
df['Cek Tipe Toko'] = np.where(df['Type toko'] == 'Kios Pulsa','Kios Pulsa', df['Cek Tipe GT'])
df['Cek Tipe Toko']

0       Kios Pulsa
1       Kios Pulsa
2       Kios Pulsa
3         GT Small
4       Kios Pulsa
           ...    
3037      GT Large
3038    Kios Pulsa
3039    Kios Pulsa
3040      GT Small
3041    Kios Pulsa
Name: Cek Tipe Toko, Length: 3042, dtype: object

In [126]:
df.pivot_table(index=['PROVINCE','KOTA/KABUPATEN'],
               values= ['RESPONSE_ID'],
               columns= ['Cek Tipe Toko'],
               aggfunc= 'count')

RESPONSE_ID                              
Cek Tipe Toko                         GT Large GT Small GT Warung Kios Pulsa
PROVINCE           KOTA/KABUPATEN                                           
DI YOGYAKARTA      YOGYAKARTA            16.00    20.00     16.00      19.00
DKI JAKARTA        JAKARTA BARAT         26.00    49.00       nan      88.00
                   JAKARTA PUSAT         15.00    13.00       nan      35.00
                   JAKARTA SELATAN       11.00    42.00      2.00      57.00
                   JAKARTA TIMUR         30.00   149.00      2.00     204.00
                   JAKARTA UTARA         34.00    61.00      2.00      81.00
JAWA BARAT         BANDUNG               40.00   307.00      9.00     108.00
                   CIREBON               67.00    81.00     21.00      15.00
                   KARAWANG              44.00    91.00      7.00     100.00
                   SUKABUMI              24.00   155.00      6.00      16.00
JAWA TIMUR         SURABAYA              29.00    85.00      7.00     123.00
KALIMANTAN SELATAN BANJARMASIN            8.00    21.00      3.00      32.00
LAMPUNG            BANDAR LAMPUNG        14.00    28.00      1.00      43.00
RIAU               PEKANBARU             12.00    25.00      7.00      41.00
SULAWESI SELATAN   MAKASSAR              11.00    63.00      6.00      56.00
SULAWESI UTARA     MANADO                 8.00    35.00       nan      20.00
SUMATERA SELATAN   PALEMBANG             13.00    34.00      8.00      73.00
SUMATERA UTARA     MEDAN                 24.00    51.00      9.00      89.00

In [127]:
conditions = [
    (df['OMSET PER BULAN'] > 10000000),
    (df['OMSET PER BULAN'] <= 10000000)]
              
values = ['GT Large', 'GT Small']

df['Cek Tipe GT'] = np.select(conditions, values)


In [128]:
# Cek Tipe Toko (GT Large/ GT Small/ Kios Pulsa)
df['Cek Tipe Toko'] = np.where(df['Type toko'] == 'Kios Pulsa','Kios Pulsa', df['Cek Tipe GT'])
df['Cek Tipe Toko']

0       Kios Pulsa
1       Kios Pulsa
2       Kios Pulsa
3         GT Small
4       Kios Pulsa
           ...    
3037      GT Large
3038    Kios Pulsa
3039    Kios Pulsa
3040      GT Small
3041    Kios Pulsa
Name: Cek Tipe Toko, Length: 3042, dtype: object

In [129]:
df.pivot_table(index=['PROVINCE','KOTA/KABUPATEN'],
               values= ['RESPONSE_ID'],
               columns= ['Cek Tipe Toko'],
               aggfunc= 'count')

RESPONSE_ID                    
Cek Tipe Toko                         GT Large GT Small Kios Pulsa
PROVINCE           KOTA/KABUPATEN                                 
DI YOGYAKARTA      YOGYAKARTA               16       36         19
DKI JAKARTA        JAKARTA BARAT            26       49         88
                   JAKARTA PUSAT            15       13         35
                   JAKARTA SELATAN          11       44         57
                   JAKARTA TIMUR            30      151        204
                   JAKARTA UTARA            34       63         81
JAWA BARAT         BANDUNG                  40      316        108
                   CIREBON                  67      102         15
                   KARAWANG                 44       98        100
                   SUKABUMI                 24      161         16
JAWA TIMUR         SURABAYA                 29       92        123
KALIMANTAN SELATAN BANJARMASIN               8       24         32
LAMPUNG            BANDAR LAMPUNG           14       29         43
RIAU               PEKANBARU                12       32         41
SULAWESI SELATAN   MAKASSAR                 11       69         56
SULAWESI UTARA     MANADO                    8       35         20
SUMATERA SELATAN   PALEMBANG                13       42         73
SUMATERA UTARA     MEDAN                    24       60         89

# Ukuran Jalan Toko/ Warung
Rata2 toko berada di jalan 1 atau 2 mobil

In [43]:
df.groupby('Ukuran Jalan').agg('count')['RESPONSE_ID']/df.shape[0]*100

Ukuran Jalan
1 Mobil              36.95
1 Motor               7.56
2 Mobil              46.35
Lebih Dari 2 Mobil    9.14
Name: RESPONSE_ID, dtype: float64

# Ada/ Tidak ada Papan Nama Toko

In [44]:
df.groupby('Ada Papan Nama Toko').agg('count')['RESPONSE_ID']
#/df.shape[0]*100

Ada Papan Nama Toko
Ada          1688
Tidak Ada    1354
Name: RESPONSE_ID, dtype: int64

In [45]:
df[df['Ada Papan Nama Toko'] == 'Ada'].groupby('Branding Platform Online di Nama Toko').agg('count')['RESPONSE_ID']
#/df.groupby('Ada Papan Nama Toko').agg('count')['RESPONSE_ID'][0]*100

Branding Platform Online di Nama Toko
Ada           353
Tidak Ada    1335
Name: RESPONSE_ID, dtype: int64

In [46]:
df.iloc[:,11:47] = df.iloc[:,11:47].dropna().astype(int)

# Branding Platform  Toko

In [47]:
df.pivot_table(values= ['Branding_Ayo SRC', 'Branding_Ayopop', 'Branding_BCA M-Banking',
               'Branding_Blibli Mitra', 'Branding_BNI/Syariah Mobile Banking', 'Branding_BNI46',
               'Branding_BRIlink', 'Branding_Dana', 'Branding_Doku',
               'Branding_Flip', 'Branding_Gopay', 'Branding_GrabKios',
               'Branding_Gudang Pulsa', 'Branding_GudangAda', 'Branding_Indopay',
               'Branding_iSaku', 'Branding_JakOne Mobile', 'Branding_Kioser',
               'Branding_LangitPay', 'Branding_LinkAja', 'Branding_Mitra Bukalapak',
               'Branding_GoToko', 'Branding_Mitra Shopee', 'Branding_Mitra Tokopedia',
               'Branding_Mkios', 'Branding_MPI Reload', 'Branding_OCTO Go Mobile',
               'Branding_Ovo', 'Branding_OBA ( Outlet Binaan Alfamart)', 'Branding_Payfazz',
               'Branding_Paytren', 'Branding_PermataMe', 'Branding_Propana Reload',
               'Branding_Sakuku', 'Branding_Ula', 'Branding_Warung Pintar'],
               columns= 'Branding Platform Online di Nama Toko' ,aggfunc= 'sum'
               ).sort_values('Ada', ascending= False)
               #/df[df['Ada Papan Nama Toko'] == 'Ada'].groupby('Branding Platform Online di Nama Toko').agg('count')['RESPONSE_ID'][0]*100

Branding Platform Online di Nama Toko,Ada,Tidak Ada
Branding_Ayo SRC,122.00,0.00
Branding_BRIlink,96.00,0.00
Branding_Mitra Bukalapak,45.00,0.00
Branding_Mitra Tokopedia,29.00,0.00
Branding_Mitra Shopee,24.00,0.00
Branding_LinkAja,19.00,0.00
Branding_BNI46,15.00,0.00
Branding_Gopay,13.00,0.00
Branding_Ovo,13.00,0.00
Branding_Blibli Mitra,10.00,0.00


# Dual Bisnis atau tidak
Hanya 12,5% Toko yang menjalankan Dual Bisnis

In [48]:
df.groupby('Dual Bisnis').agg('count')['RESPONSE_ID']
#/df.shape[0]*100

Dual Bisnis
Tidak    2663
Ya        379
Name: RESPONSE_ID, dtype: int64

# Rentang usia OWNER


> Sebanyak 65% dari sample, usia owner berada di kisaran 31-50 tahun





In [130]:
df.groupby('Usia OWNER').agg('count')['RESPONSE_ID']
#/df.shape[0]*100

Usia OWNER
0 - 10        1
11 - 20      19
21 - 30     511
31 - 40    1129
41 - 50     857
51 - 60     419
61 - 70      96
71 - 80       9
81 - 90       1
Name: RESPONSE_ID, dtype: int64

# Seberapa welcome para pemilik toko terhadap sensus ini

> 99,9% Toko mengijikan auditor untuk mengambil data sensus



In [50]:
df.groupby('Apakah Auditor diizinkan untuk mendapatkan informasi mengenai toko untuk sensus ini?').agg('count')['RESPONSE_ID']/df.shape[0]*100


Apakah Auditor diizinkan untuk mendapatkan informasi mengenai toko untuk sensus ini?
Iya (Diperbolehkan)   99.87
Menolak                0.13
Name: RESPONSE_ID, dtype: float64

# Keinginan Menjadi Mitra Bukalapak

In [131]:
df.groupby('Ingin jadi Mitra Bukalapak').agg('count')['RESPONSE_ID']
#/df.shape[0]*100

Ingin jadi Mitra Bukalapak
Tidak   66.67
Ya      23.90
Name: RESPONSE_ID, dtype: float64